In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only allocate 2GB of memory on the first GPU
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=2048)]
        )
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)


1 Physical GPUs, 1 Logical GPUs


In [2]:
from keras.utils import image_dataset_from_directory
import tensorflow as tf


with tf.device('/device:GPU:0'):
    train_dir_1 = 'trainning/train1'
    train_dir_2 = 'trainning/train2'
    train_dir_3 = 'trainning/train3'
    validation_dir = 'train4' # Validation
    train_dir_5 = 'trainning/train5'
    test_dir = 'test'

    trainning = [train_dir_1, train_dir_2,train_dir_3,train_dir_5]

    train_dir = train_dir_2
    IMG_SIZE = 150  # InceptionResNetV2 requires 299x299 images

    # image_dataset_from_directory with labels="inferred" for 
    # getting the images in the subdirectories and translating the subdirectory as a class 
    # of type categorical
    #train_dataset = image_dataset_from_directory(train_dir,image_size=(IMG_SIZE, IMG_SIZE),batch_size=32, labels="inferred", label_mode="categorical")
    test_dataset = image_dataset_from_directory(test_dir,image_size=(IMG_SIZE, IMG_SIZE), labels="inferred",label_mode="categorical")
    validation_dataset = image_dataset_from_directory(validation_dir,image_size=(IMG_SIZE, IMG_SIZE), labels="inferred",label_mode="categorical")

    train_dataset = tf.data.Dataset

    for i in trainning:
        if i == trainning[0]:
            train_dataset = image_dataset_from_directory(i, image_size=(IMG_SIZE, IMG_SIZE), labels="inferred", label_mode="categorical")
            continue
        train_dataset = train_dataset.concatenate( image_dataset_from_directory(i, image_size=(IMG_SIZE, IMG_SIZE), labels="inferred", label_mode="categorical"))




Found 10000 files belonging to 10 classes.
Found 10000 files belonging to 10 classes.
Found 10000 files belonging to 10 classes.
Found 10000 files belonging to 10 classes.
Found 10000 files belonging to 10 classes.
Found 10000 files belonging to 10 classes.


In [3]:
import matplotlib.pyplot as plt

def graph(history):
    # Use the correct key names from the history object
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc) + 1)

    plt.figure(figsize=(14, 5))

    # Plot training and validation accuracy
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, 'bo-', label='Training accuracy')
    plt.plot(epochs, val_acc, 'b-', label='Validation accuracy')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot training and validation loss
    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, 'bo-', label='Training loss')
    plt.plot(epochs, val_loss, 'b-', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()


In [ ]:
import tensorflow as tf
import numpy as np
from keras.applications.densenet import DenseNet121, preprocess_input
NUM_CLASSES = 10
# Define base model
base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))

def get_features_and_labels(dataset):
    tf.keras.backend.clear_session()

    all_features = []
    all_labels = []
    for images, labels in dataset:
        preprocessed_images = preprocess_input(images)
        features = base_model.predict(preprocessed_images)
        all_features.append(features)
        all_labels.append(labels)
    return np.concatenate(all_features), np.concatenate(all_labels)
with tf.device('/gpu:0'):
    train_features, train_labels = get_features_and_labels(train_dataset)
    val_features, val_labels = get_features_and_labels(validation_dataset)
    test_features, test_labels = get_features_and_labels(test_dataset)

    # Convert the features and labels into tf.data.Dataset
    train_dataset = tf.data.Dataset.from_tensor_slices((train_features, train_labels))
    val_dataset = tf.data.Dataset.from_tensor_slices((val_features, val_labels))
    test_dataset = tf.data.Dataset.from_tensor_slices((test_features, test_labels))


In [ ]:
np.save('train_features.npy', train_features)
np.save('val_features.npy', val_features)
np.save('test_features.npy', test_features)
np.save('train_labels.npy', train_labels)
np.save('val_labels.npy', val_labels)
np.save('test_labels.npy', test_labels)

In [ ]:
from tensorflow.keras.utils import Sequence

class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

In [63]:

from numpy import load
import numpy as np
train_features = load('train_features.npy')
val_features = load('val_features.npy')
test_features = load('test_features.npy')
train_labels = load('train_labels.npy')
val_labels = load('val_labels.npy')
test_labels = load('test_labels.npy')

train_gen = DataGenerator(train_features, train_labels, 32)
val_gen = DataGenerator(val_features, val_labels, 32)
test_gen = DataGenerator(test_features, test_labels, 32)


In [ ]:
import optuna
import keras
from keras import layers
import tensorflow as tf
import numpy as np

with tf.device('/device:GPU:0'):

    def create_model(trial):
        keras.backend.clear_session()
        # Define the regularization and dropout values
        reg = keras.regularizers.l2(0.005)
        dropFinal = 0.5

        # Add custom top layers
        #inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
        #x = base_model(inputs, training=False)
        inputs = keras.Input(shape=(4, 4, 1024))
        x = layers.Flatten()(inputs)
        x = layers.Dense(trial.suggest_int('dense_units', 256, 512, step=256), activation='relu', kernel_regularizer=reg)(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(dropFinal)(x)
        outputs = layers.Dense(10, activation='softmax')(x)
        
        model = keras.Model(inputs, outputs)
        
        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate=trial.suggest_float('learning_rate_IN', 1e-3, 1e-2,log=True),
            decay_steps=1000,
            decay_rate=0.6,
            staircase=True
        )

        optimizer_options = ['Adam', 'RMSprop']
        optimizer_selected = trial.suggest_categorical('optimizer', optimizer_options)
        if optimizer_selected == 'Adam':
            optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)
        else:
            optimizer = keras.optimizers.RMSprop(learning_rate=lr_schedule)

        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        
        return model

    def objective(trial):
        model = create_model(trial)
        
        callbacks = [
            keras.callbacks.ModelCheckpoint(
                filepath='models_T/model_best.h5',
                save_best_only=True,
                monitor='val_loss',
                mode='min',
                verbose=0
            ),
            keras.callbacks.EarlyStopping(
                monitor='val_loss',
                min_delta=0,
                patience=4,
                verbose=0,
                mode='min',
                restore_best_weights=True,
            ),
            keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=5,
                verbose=0
            )
        ]
        batch_size = trial.suggest_int('batch_size', 8, 16, step=8)
        
        history = model.fit(
            train_gen,
            batch_size=batch_size,
            epochs=50,
            validation_data=val_gen,
            callbacks=callbacks,
            verbose=0
        )
        
        val_loss = history.history['val_loss'][-1]
        return val_loss




In [ ]:
with tf.device('/device:GPU:0'):

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=5)

    print("Best trial:")
    trial = study.best_trial

    print(f"Value: {trial.value}")
    print("Params:")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")

In [4]:
import keras
# Load do novo melhor modelo
keras.backend.clear_session()

model = keras.models.load_model("models_T\model_best_DA.h5")
#model = create_model(study.best_trial)

In [ ]:

with tf.device('/device:GPU:0'):

    callbacks = [
            keras.callbacks.ModelCheckpoint(
                filepath='models_T/model_best_First.h5',
                save_best_only=True,
                monitor='val_loss',
                mode='min',
                verbose=0
            ),
            keras.callbacks.EarlyStopping(
                monitor='val_loss',
                min_delta=0,
                patience=4,
                verbose=0,
                mode='min',
                restore_best_weights=True,
            )
        ]
    
    history = model.fit(
            train_gen,
            batch_size=16,
            epochs=100,
            validation_data=val_gen,
            callbacks=callbacks,
            verbose=1
        )
    
    graph(history)

In [10]:

val_loss, val_acc = model.evaluate(test_dataset) #test_gen
print('Test_acc:', val_acc,'\nTest_loss:', val_loss)

#keras.models.save_model(model,'models_T/model_best_First.h5')

161/313 [==============>...............] - ETA: 3s - loss: 37.4032 - accuracy: 0.3853

KeyboardInterrupt: 

In [6]:
from keras.applications.densenet import DenseNet121
# Define base model
base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False

In [8]:
import optuna
import keras
from keras import layers
import tensorflow as tf
import numpy as np
from keras.applications.densenet import preprocess_input
with tf.device('/device:GPU:0'):

    def create_model(trial):
        keras.backend.clear_session()
        # Define the regularization and dropout values
        reg = keras.regularizers.l2(0.005)
        dropFinal = 0.5
        data_augmentation = keras.Sequential(
            [
            layers.RandomFlip("horizontal"),
            layers.RandomRotation(0.1),
            layers.RandomZoom(0.2),
            ]
            )

        # Add custom top layers
        inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
        x = data_augmentation(inputs)
        x = preprocess_input(x)
        x = base_model(x)
        x = layers.Flatten()(inputs)
        x = layers.Dense(trial.suggest_int('dense_units', 256, 512, step=256), activation='relu', kernel_regularizer=reg)(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(dropFinal)(x)
        outputs = layers.Dense(10, activation='softmax')(x)
        
        model = keras.Model(inputs, outputs)
        
        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate=trial.suggest_float('learning_rate_IN', 1e-3, 1e-2,log=True),
            decay_steps=1000,
            decay_rate=0.6,
            staircase=True
        )

        optimizer_options = ['Adam', 'RMSprop']
        optimizer_selected = trial.suggest_categorical('optimizer', optimizer_options)
        if optimizer_selected == 'Adam':
            optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)
        else:
            optimizer = keras.optimizers.RMSprop(learning_rate=lr_schedule)

        model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
        return model

    def objective1(trial):
        model = create_model(trial)
        
        callbacks = [
            keras.callbacks.ModelCheckpoint(
                filepath='models_T/model_best_DA.h5',
                save_best_only=True,
                monitor='val_loss',
                mode='min',
                verbose=0
            ),
            keras.callbacks.EarlyStopping(
                monitor='val_loss',
                min_delta=0,
                patience=4,
                verbose=0,
                mode='min',
                restore_best_weights=True,
            )
        ]
        batch_size = trial.suggest_int('batch_size', 8, 16, step=8)
        
        history = model.fit(
            train_dataset,
            batch_size=batch_size,
            epochs=20,
            validation_data=validation_dataset,
            callbacks=callbacks,
            use_multiprocessing=False,
            verbose=0
        )
        
        val_loss = history.history['val_loss'][-1]
        graph(history)
        return val_loss




In [9]:
with tf.device('/device:GPU:0'):

    study = optuna.create_study(direction='minimize')
    study.optimize(objective1, n_trials=5)

    print("Best trial:")
    trial = study.best_trial

    print(f"Value: {trial.value}")
    print("Params:")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")

[I 2024-06-20 21:44:06,915] A new study created in memory with name: no-name-6bc3fedc-04f2-45cb-94a5-334b28254e11


[W 2024-06-20 21:49:49,713] Trial 0 failed with parameters: {'dense_units': 256, 'learning_rate_IN': 0.007931875652418725, 'optimizer': 'RMSprop', 'batch_size': 8} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\HP\.conda\envs\tensorflow_gpu\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\HP\AppData\Local\Temp\ipykernel_19724\1132962580.py", line 74, in objective1
    history = model.fit(
  File "c:\Users\HP\.conda\envs\tensorflow_gpu\lib\site-packages\keras\utils\traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
  File "c:\Users\HP\.conda\envs\tensorflow_gpu\lib\site-packages\keras\engine\training.py", line 1555, in fit
    for step in data_handler.steps():
  File "c:\Users\HP\.conda\envs\tensorflow_gpu\lib\site-packages\keras\engine\data_adapter.py", line 1374, in steps
    original_spe = self._steps_per_execution.numpy().item()

KeyboardInterrupt: 